In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load all CSV files
customers = pd.read_csv("olist_customers_dataset.csv")
geolocation = pd.read_csv("olist_geolocation_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
order_payments = pd.read_csv("olist_order_payments_dataset.csv")
order_reviews = pd.read_csv("olist_order_reviews_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
product_category = pd.read_csv("product_category_name_translation.csv")

# Function to clean a dataframe
def clean_df(df):
    df = df.drop_duplicates()  # Remove duplicate rows
    df = df.dropna(axis=0, how='all')  # Remove rows where all values are NaN
    df = df.dropna(axis=1, how='all')  # Remove columns where all values are NaN
    df = df.fillna(method='ffill').fillna(method='bfill')  # Fill missing values
    return df

# Clean all datasets
customers = clean_df(customers)
geolocation = clean_df(geolocation)
order_items = clean_df(order_items)
order_payments = clean_df(order_payments)
order_reviews = clean_df(order_reviews)
orders = clean_df(orders)
products = clean_df(products)
sellers = clean_df(sellers)
product_category = clean_df(product_category)

# Verify cleanup results
print("Customers missing values:\n", customers.isnull().sum(), "\n")
print("Geolocation missing values:\n", geolocation.isnull().sum(), "\n")
print("Order Items missing values:\n", order_items.isnull().sum(), "\n")
print("Order Payments missing values:\n", order_payments.isnull().sum(), "\n")
print("Order Reviews missing values:\n", order_reviews.isnull().sum(), "\n")
print("Orders missing values:\n", orders.isnull().sum(), "\n")
print("Products missing values:\n", products.isnull().sum(), "\n")
print("Sellers missing values:\n", sellers.isnull().sum(), "\n")
print("Product Category missing values:\n", product_category.isnull().sum(), "\n")

#Converting columns into datetimes and converting their data types
# Convert date columns into datetime format
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_columns:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# Create a new column: delivery_time = delivered date - purchase date
orders['delivery_time'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days

# Extract year and month from purchase date
orders['purchase_year'] = orders['order_purchase_timestamp'].dt.year
orders['purchase_month'] = orders['order_purchase_timestamp'].dt.month

# Display a sample to verify
print(orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 
              'delivery_time', 'purchase_year', 'purchase_month']].head())

#Now Data Analysis
# Merge all necessary datasets first
merged_df = (orders
             .merge(order_items, on='order_id', how='left')
             .merge(products, on='product_id', how='left')
             .merge(order_reviews[['order_id', 'review_score']], on='order_id', how='left')
             .merge(customers, on='customer_id', how='left')
             .merge(order_payments, on='order_id', how='left'))

# Add translated product categories
merged_df = merged_df.merge(product_category, on='product_category_name', how='left')

# Find which product categories bring the most revenue
revenue_by_category = (merged_df.groupby('product_category_name_english')['price']
                       .sum()
                       .sort_values(ascending=False)
                       .reset_index())

print("Top 10 Product Categories by Revenue:")
print(revenue_by_category.head(10))
print("="*70)

# Find which cities or states have the highest number of orders
orders_by_city = (merged_df.groupby('customer_city')['order_id']
                  .nunique()
                  .sort_values(ascending=False)
                  .reset_index())

orders_by_state = (merged_df.groupby('customer_state')['order_id']
                   .nunique()
                   .sort_values(ascending=False)
                   .reset_index())

print("Top 10 Cities by Number of Orders:")
print(orders_by_city.head(10))
print("\nTop 10 States by Number of Orders:")
print(orders_by_state.head(10))
print("="*70)

# Check how delivery time affects review scores
merged_df['delivery_time'] = (merged_df['order_delivered_customer_date'] - merged_df['order_purchase_timestamp']).dt.days
delivery_vs_review = merged_df.groupby('review_score')['delivery_time'].mean().reset_index()

print("Average Delivery Time vs Review Score:")
print(delivery_vs_review)
print("="*70)

# Find how many repeat customers there are
repeat_customers = merged_df.groupby('customer_unique_id')['order_id'].nunique()
repeat_count = (repeat_customers > 1).sum()
total_customers = repeat_customers.count()
repeat_percentage = (repeat_count / total_customers) * 100

print(f"Total Customers: {total_customers}")
print(f"Repeat Customers: {repeat_count} ({repeat_percentage:.2f}%)")
print("="*70)

# Calculate total and average revenue per customer
revenue_per_customer = (merged_df.groupby('customer_unique_id')['price']
                        .sum()
                        .reset_index()
                        .rename(columns={'price': 'total_revenue'}))
revenue_per_customer['avg_revenue_per_order'] = revenue_per_customer['total_revenue'] / repeat_customers.values

print("Sample of Revenue per Customer:")
print(revenue_per_customer.head())

merged_df.to_csv("Olist_Ecommerce_Analysis.csv", index=False)
print("✅ Cleaned and combined dataset saved as 'Olist_Ecommerce_Analysis.csv'")

C:\Users\hp\AppData\Local\Temp\ipykernel_23272\734082475.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')  # Fill missing values
C:\Users\hp\AppData\Local\Temp\ipykernel_23272\734082475.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')  # Fill missing values
C:\Users\hp\AppData\Local\Temp\ipykernel_23272\734082475.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')  # Fill missing values
C:\Users\hp\AppData\Local\Temp\ipykernel_23272\734082475.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use

Customers missing values:
 customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64 

Geolocation missing values:
 geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64 

Order Items missing values:
 order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64 

Order Payments missing values:
 order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64 

Order Reviews missing values:
 review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       